In [7]:
from dotenv import load_dotenv

load_dotenv()

import httpx
import duckdb
import json
from src.settings import settings
from datetime import datetime
from datetime import datetime
from src.assets.gigcal.schemas import Post, AccountPlatform

In [9]:
%load_ext sql
conn = duckdb.connect(settings.DUCKDB_DATABASE)
conn.sql("SET search_path TO ae_de_play;")
conn.install_extension('spatial')
conn.load_extension('spatial')
conn.install_extension('delta')
conn.load_extension('delta')

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [2]:
retrieved_at = datetime.now()

In [3]:
with httpx.Client(
    base_url="https://instagram-social-api.p.rapidapi.com",
) as client:
    response = client.get(
        "/v1/posts",
        headers={
            "x-rapidapi-host": "instagram-social-api.p.rapidapi.com",
            "x-rapidapi-key": settings.RAPIDAPI_INSTAGRAM_API_KEY.get_secret_value(),
        },
        params={
            "username_or_id_or_url": "sarisaribar"
        },
    )
    posts = response.json()

In [4]:
print(json.dumps(posts['data']['items'], indent=2, ensure_ascii=False))

[
  {
    "all_previous_submitters": [],
    "boost_unavailable_identifier": null,
    "boost_unavailable_reason": null,
    "boost_unavailable_reason_v2": null,
    "can_modify_carousel": true,
    "can_reply": false,
    "can_reshare": true,
    "can_save": true,
    "caption": {
      "created_at": 1756725390,
      "created_at_utc": 1756725390,
      "did_report_as_spam": false,
      "hashtags": [
        "#SariSari",
        "#HiddenBar",
        "#MakatiBar",
        "#Gig",
        "#LiveEvents",
        "#BarReccomendation",
        "#Explore",
        "#ExplorePage",
        "#OPM",
        "#LiveMusic"
      ],
      "id": "3712004065082883273",
      "is_covered": false,
      "is_ranked_comment": false,
      "mentions": [],
      "private_reply_status": 0,
      "share_enabled": false,
      "text": "Events of the week (September 1 - September 7) 🎶 \n\nPlan out your week with Sari Sari 🍻 \n\n☎️ DM us for table reservations\n\n#SariSari #HiddenBar #MakatiBar #Gig #LiveEven

In [8]:
parsed_posts = [
    Post(
        **post,
        retrieved_at=retrieved_at,
        account_platform=AccountPlatform.INSTAGRAM,
    )
    for post in posts['data']['items']
]

In [9]:
print(json.dumps([
    p.model_dump(mode='json')
    for p in parsed_posts
], indent=2, ensure_ascii=False))

[
  {
    "id": "3712004065082883273",
    "account_name": "sarisaribar",
    "account_platform": "instagram",
    "posted_at": "2025-09-01T11:16:29Z",
    "retrieved_at": "2025-09-07T20:52:35.103882",
    "caption": "Events of the week (September 1 - September 7) 🎶 \n\nPlan out your week with Sari Sari 🍻 \n\n☎️ DM us for table reservations\n\n#SariSari #HiddenBar #MakatiBar #Gig #LiveEvents #BarReccomendation #Explore #ExplorePage #OPM #LiveMusic",
    "image_url": "https://scontent-vie1-1.cdninstagram.com/v/t51.2885-15/542244010_17866455153443158_3575004617888539251_n.jpg?se=7&stp=dst-jpg_e35_tt6&efg=eyJ2ZW5jb2RlX3RhZyI6IkNBUk9VU0VMX0lURU0uaW1hZ2VfdXJsZ2VuLjEwMjR4MTI3Ni5zZHIuZjgyNzg3LmRlZmF1bHRfaW1hZ2UuYzIifQ&_nc_ht=scontent-vie1-1.cdninstagram.com&_nc_cat=110&_nc_oc=Q6cZ2QFyMpflc6lw26Q0x3Jhdotp9002yLQv5sBUJ6esKgC3qK-arT1GsOTN1pm8I2W9YuxSIrwS6YpwW13QKzzqN2j8&_nc_ohc=TPd7RInJd-kQ7kNvwHxlNyr&_nc_gid=s4QrxZ1Sk3hWBXKtNqrchA&edm=ABmJApABAAAA&ccb=7-5&ig_cache_key=MzcxMjAwNDA0ODYyNDQyNDg4MQ